In [0]:
%run "/Workspace/Users/jubin.koruthu@edhec.com/Databricks-tutorial-1/walkaround"

#DELTA LAKE

In [0]:
df_sales.write.format('delta')\
  .mode('append')\
  .option('path', 'abfss://destination@datalakejubin.dfs.core.windows.net/sales')\
  .save()

## Managed vs External Delta Tables

#### Database

In [0]:
%sql
create database saleDB

#### Managed Table

In [0]:
%sql
create table saleDB.mantable(
    id int,
    name string,
    marks int
)
using delta 

In [0]:
%sql
insert into saleDB.mantable VALUES 
(1,'aa', 30),
(2,'bb', 40),
(3,'cc', 50)

In [0]:
%sql
select * from saleDB.mantable 

In [0]:
%sql
drop table saleDB.mantable

#### External Tables

In [0]:
%sql
create table saleDB.exttable(
  id int,
  name string,
  marks int
)
using delta
location 'abfss://destination@datalakejubin.dfs.core.windows.net/saleDB/exttable' 

due to lack of entra id an external table cant be made since we need unity catalog to point to an external table backed by storage credentials which can only be made by entra id

In [0]:
%sql
SELECT * 
FROM delta.`abfss://destination@datalakejubin.dfs.core.windows.net/sales`;

#### UPDATE

In [0]:
%sql
UPDATE delta.`abfss://destination@datalakejubin.dfs.core.windows.net/sales`
SET Item_Weight = 9
WHERE Item_Identifier = 'FDA15';

In [0]:
%sql
SELECT * 
FROM delta.`abfss://destination@datalakejubin.dfs.core.windows.net/sales`
where Item_Identifier = 'FDA15'

#### Deletion

In [0]:
%sql
DELETE 
FROM delta.`abfss://destination@datalakejubin.dfs.core.windows.net/sales`
where Item_Identifier = 'FDA15' and Item_Fat_Content = 'LF'

#### DATA VERSIONING

In [0]:
%sql
describe history delta.`abfss://destination@datalakejubin.dfs.core.windows.net/sales`

#### Time travel(Restore)

In [0]:
%sql
restore table delta.`abfss://destination@datalakejubin.dfs.core.windows.net/sales` version as of 1

In [0]:
%sql
SELECT * 
FROM delta.`abfss://destination@datalakejubin.dfs.core.windows.net/sales`
where Item_Identifier = 'FDA15'

#### VACUUM( deletion of whole data from external table including metadata where its used only in rare scenarios)


vacuum delta.`abfss://destination@datalakejubin.dfs.core.windows.net/sales` (for now even if i run nothing would happen if the files are less than 7 days old)
vacuum delta.`abfss://destination@datalakejubin.dfs.core.windows.net/sales` retain o hours;(this will delete the files instantly despite it being less than 7 days old)

### Delta Table Optimization

#### Optimize(optimized the data for querying faster)

In [0]:
%sql
optimize delta.`abfss://destination@datalakejubin.dfs.core.windows.net/sales`

In [0]:
%sql
select * from delta.`abfss://destination@datalakejubin.dfs.core.windows.net/sales`

#### ZORDER BY

In [0]:
%sql
optimize delta.`abfss://destination@datalakejubin.dfs.core.windows.net/sales` zorder by (Item_Identifier)

In [0]:
%sql
select * from delta.`abfss://destination@datalakejubin.dfs.core.windows.net/sales`

#### AUTO LOADER

#### Streaming dataframe

In [0]:
df = spark.readStream.format('cloudFiles')\
  .option('cloudFiles.format', 'parquet')\
  .option('cloudFiles.schemaLocation', 'abfss://aldestination@datalakejubin.dfs.core.windows.net/checkpoint')\
  .load('abfss://alsource@datalakejubin.dfs.core.windows.net')

In [0]:
df.writeStream.format('delta')\
    .option('checkpointLocation', 'abfss://aldestination@datalakejubin.dfs.core.windows.net/checkpoint')\
    .trigger(processingTime='5 seconds')\
    .start('abfss://aldestination@datalakejubin.dfs.core.windows.net/data') ## whenever i add new data it will be appended since every 5 seconds a check is done and the new data is appended to the destination folder in container